In [1]:
import os
import numpy as np
import pandas as pf
from skimage.io import imread
from scipy.misc import imresize
import re
from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_base_path = "../dataset/DSB/train_all"
train_ids = next(os.walk(train_base_path))[1]

In [3]:
def get_masks(base_path):
    masks = dict()
    id_extractor = re.compile(f"{base_path}\{os.sep}(?P<image_id>.*)\{os.sep}masks\{os.sep}(?P<mask_id>.*)\.png")
    for mask_path in glob(os.path.join(base_path, "**", "masks", "*.png")):
        matches = id_extractor.match(mask_path)

        image_id = matches.group("image_id")
        image_file = os.listdir(os.path.join(base_path, image_id, "images"))[0]
        image_path = os.path.join(base_path, image_id, "images", image_file)

        if image_id in masks:
            masks[image_id].append(mask_path)
        else:
            masks[image_id] = [mask_path]
    return masks

In [4]:
def get_cell_pixels(masks):
    pixels = 0
    cell_num = 0
    for mask_path in masks:
        mask = imread(mask_path)
        pixels += len(np.where(mask == 255)[0])
    mean_cell = pixels/len(masks)
    return mean_cell

In [5]:
def get_mean_cell(ids, masks):
    mask_pixels = dict()
    all_cell_pixels = 0
    for id in tqdm(ids):
        mask_pixels[id] = get_cell_pixels(masks[id])
        all_cell_pixels += mask_pixels[id]
    mean_cell = all_cell_pixels/len(ids)
    return mean_cell, mask_pixels

In [6]:
train_masks = get_masks(train_base_path)
train_mean_cell, mask_pixels = get_mean_cell(train_ids, train_masks)

100%|██████████| 694/694 [04:46<00:00,  2.42it/s]


In [7]:
train_mean_cell

559.0373166257757

In [9]:
mask_pixels[train_ids[1]]

1294.4

In [23]:
tmp = imread(train_masks[train_ids[1]][0])
tmp.shape

(360, 360)

In [24]:
new_w = int(tmp.shape[1] * (train_mean_cell*2/mask_pixels[train_ids[1]]))
new_h = int(tmp.shape[0] * (train_mean_cell*2/mask_pixels[train_ids[1]]))
tmp = imresize(tmp, (new_w, new_h))

/root/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


In [25]:
tmp.shape

(310, 310)